In [1]:
import os
import json
import numpy as np
from subgraph_utils import *
import pandas as pd
from utils import *
from utils import GraphAlgos

/apps/conda/isha/envs/quacer-b/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
prime_folder = '/share/quacer_c/prime'
actual_rels = json.load(open(os.path.join(prime_folder, 'actual_rels.json')))
edge2src = json.load(open(os.path.join(prime_folder, 'edge2src.json')))
graph = json.load(open(os.path.join(prime_folder, 'graph.json')))
id2name = json.load(open(os.path.join(prime_folder, 'id2name.json')))
id2source = json.load(open(os.path.join(prime_folder, 'id2source.json')))
rels = json.load(open(os.path.join(prime_folder, 'rels.json')))
graph_text_edge = json.load(open(os.path.join(prime_folder, 'graph_text_edge.json')))

In [3]:
entity_aliases = {}
relation_aliases = {}
for k, v in id2name.items():
    if k.startswith('RIDR'):
        relation_aliases[k] = [v]
    else:
        entity_aliases[k] = [v]

In [4]:
graph_algos = GraphAlgos(graph, entity_aliases, relation_aliases)

In [5]:
from discovery_functions import least_side_effects_discoverer

discovery_funcs = [least_side_effects_discoverer]

query_generator = CustomQueryGenerator(graph_algos, discovery_funcs)

In [13]:
query_results = query_generator.get_single_query(0, id2name=id2name)
query_data = query_generator.generate_query_data(query_results, id2name=id2name, 
                                                 graph_text_edge=graph_text_edge, graph_text_sentencized=None,
                                                 tokenizer=None)

In [14]:
query_data

{'query': 'Which drug used to treat sickle cell anemia has the least number of side effects?',
 'correct_answers': ['Voxelotor'],
 'path_id': ['ENID15747', 'ENID28921'],
 'path_en': ['Voxelotor', 'sickle cell anemia'],
 'context': 'obsolete neurogenic bladder (disease) is related to Oxybutynin by indication_drug obsolete neurogenic bladder (disease) is related to Butacaine by indication_drug Oxybutynin is related to Abdominal pain by side effect_effect/phenotype Oxybutynin is related to Airway obstruction by side effect_effect/phenotype Oxybutynin is related to Anaphylactic shock by side effect_effect/phenotype Oxybutynin is related to Poor appetite by side effect_effect/phenotype Oxybutynin is related to Anxiety by side effect_effect/phenotype Oxybutynin is related to Arrhythmia by side effect_effect/phenotype Oxybutynin is related to Arthralgia by side effect_effect/phenotype Oxybutynin is related to Arthritis by side effect_effect/phenotype Oxybutynin is related to Back pain by side